In [1]:
import hsfs
from hsfs import engine
import json

import hopsworks

# connect to hopsworks
project = hopsworks.login()
fs = project.get_feature_store()



Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
3,application_1707759051297_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://hopsworks0.logicalclocks.com/p/119
Connected. Call `.close()` to terminate connection gracefully.

In [2]:

smartwatch_fg = fs.get_or_create_feature_group(
    name="smartwatch_sensor",
    version=1,
    description="heart rate ppg",
    primary_key=['index'],
    stream=True,
    online_enabled=True
)

In [3]:
connector = fs.get_storage_connector("dj_kafka")

config = connector.spark_options()
config

{'kafka.sasl.jaas.config': "org.apache.kafka.common.security.plain.PlainLoginModule required username='7T3PK6VD5536CRZT' password='8g3AylQQHno81xAmJPQIvdICWJsw19i6QCtZM9sCTz9BUj4y/+rxfSM7CQUePaJT';", 'kafka.sasl.mechanism': 'PLAIN', 'kafka.startingOffsets': 'earliest', 'kafka.bootstrap.servers': 'pkc-l7q2j.europe-north1.gcp.confluent.cloud:9092', 'kafka.security.protocol': 'SASL_SSL', 'kafka.ssl.endpoint.identification.algorithm': ''}

In [4]:
# create kafka topic
KAFKA_TOPIC_NAME = "topic_smartwatch_data"

df_read = spark \
    .readStream \
    .format("kafka") \
    .options(**config) \
    .option("startingOffsets", "earliest") \
    .option("subscribe", KAFKA_TOPIC_NAME) \
    .load()

In [5]:
from pyspark.sql.types import (
    LongType,
    DoubleType,
    StringType,
    TimestampType,
    StructType,
    BooleanType,
    StructField,
    IntegerType
)

from pyspark.sql.functions import from_json

parse_schema = StructType([StructField('index', IntegerType(), True),
                           StructField('ppg', LongType(), True),
                           StructField('timer', StringType(), True)
                           ])


In [6]:
df_read

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [7]:
transaction_streaming_df = df_read.selectExpr("CAST(value AS STRING)") \
     .select(from_json("value", parse_schema).alias("value")) \
     .select("value.ppg", "value.timer","value.index") \
     .selectExpr("CAST(ppg as long)", "CAST(timer as string)","CAST(index as int)" )

In [8]:
transaction_streaming_df

DataFrame[ppg: bigint, timer: string, index: int]

In [10]:
q = smartwatch_fg.insert_stream(transaction_streaming_df)

Feature Group created successfully, explore it at 
https://hopsworks0.logicalclocks.com/p/119/fs/67/fg/19

In [31]:
q.status

{'message': 'Getting offsets from KafkaV2[Subscribe[topic_smartwatch_data]]', 'isDataAvailable': False, 'isTriggerActive': True}

In [33]:
rows = smartwatch_fg.read(online=True)
rows.count()


100000